In [4]:
# imports
import os
import sys
sys.path.append("../")
import numpy as np
import umap
from functions_EDX import *
import time
import matplotlib.pyplot as plt
from scipy.stats import zscore
import hdbscan
from datetime import datetime
import seaborn as sns
from matplotlib import cm
from sklearn.preprocessing import MinMaxScaler
from skimage.feature import peak_local_max
from VCA import *
import random
import tifffile as tf
import os

## Convenient things

In [5]:
res_xy = 1024

## Locate the processed tile folder

In [6]:
spectrum_folder = '../../../primary_data/main_mosaic_6by5/Spectrum2D_PC10'
file_names = []
tmp = os.listdir(spectrum_folder)
for filename in tmp:
    if filename.endswith('.npz'):
        file_names.append(filename)

file_names = sorted(file_names)
files = [os.path.join(spectrum_folder,file_name) for file_name in file_names]

## Parameters for varying the embeddings

In [9]:
chosen_tiles = [0,5,10]
chosen_metrics = ['chebyshev','euclidean','cosine']

In [16]:
for tile_idx in chosen_tiles:
    for umetric in chosen_metrics:
        spectrum_2D = np.load(files[tile_idx])['spectrum_2D']
        
        # get a subset
        random.seed(100)
        nTrain_umap = int(0.2*res_xy**2)
        n_sample_train = np.random.choice(spectrum_2D.shape[0], nTrain_umap)
        spectrum_2D_zscored = zscore(spectrum_2D)
        
        # Train umap and fit
        start = datetime.now()
        umap1 = umap.UMAP(n_components=2, n_neighbors=10, min_dist=0.0, metric=umetric,random_state=100)
        embeddings = umap1.fit_transform(spectrum_2D_zscored[n_sample_train,:])

        end = datetime.now()
        
        # if 3 dimensions for UMAP, which ones to use (0,1,or 2)
        dim_a = 0
        dim_b = 1

        # scaling
        scaler = MinMaxScaler()
        embeddings = scaler.fit_transform(embeddings)
        etime = end-start
        print('Duration for UMAP fitting: {}'.format(end - start))
        
        
        #global threshold_rel, min_distance
        Grid = 256
        threshold_rel = 0.35
        min_distance = 20


        #find local maxima in 2D histogram
        xmin, xmax = np.amin(embeddings[:,dim_a]), np.amax(embeddings[:,dim_a])
        ymin, ymax = np.amin(embeddings[:,dim_b]), np.amax(embeddings[:,dim_b])
        counts, xedges, yedges = np.histogram2d(embeddings[:,dim_a], embeddings[:,dim_b], bins=Grid)

        local_maxi = peak_local_max(counts, threshold_rel = threshold_rel, min_distance = min_distance, exclude_border = False)
        local_max_x = []
        local_max_y = []

        for i in range(0,len(local_maxi)):
            NewValue_X = (((local_maxi[i][0] ) * (xmax - xmin)) / (Grid - 0)) + xmin
            NewValue_Y = (((local_maxi[i][1] - 0) * (ymax - ymin)) / (Grid - 0)) + ymin
            local_max_x.append(NewValue_X)
            local_max_y.append(NewValue_Y)

        x = [x for x in zip(local_max_x,local_max_y)]
        nEM = len(x)

        print(nEM)

        # set the color map according to the number of endmembers
        cmap = plt.get_cmap('nipy_spectral')
        cmap_idx = np.linspace(0, 0.9, nEM)
        
        
        min_max = 5*(1/Grid)         #determines bounding box size
        points = len(x)
        centroid = []                #empty array for selected UMAP centroids
        F = []                       #indicies of UMAP points

        for i in range(0,points):
            #set bounding square 
            Minx = x[i][0] - min_max
            Maxx = x[i][0] + min_max
            Miny = x[i][1] - min_max
            Maxy = x[i][1] + min_max

            #select points
            z3 = [0 if j == 0 else j for j in [j if j <= Maxx else 0 for j in [j if j >= Minx else 0 for j in embeddings[:,dim_a]]]]
            zz3 = [0 if j == 0 else j for j in [j if j <= Maxy else 0 for j in [j if j >= Miny else 0 for j in embeddings[:,dim_b]]]]
            f = [i for i, e in enumerate([x*y for x,y in zip(z3,zz3)]) if e != 0]
            d = embeddings[f]
            F.append(f)
            centroid.append(d)

        fig, ax = plt.subplots(figsize=(10,10))
        sns.despine(left=True, bottom=True, right=True)
        
        
        # Color Scatter with sampled pixels
        ax.scatter(embeddings[:,dim_a],embeddings[:,dim_b],c='k',s=0.01, alpha=0.1)
        ax.set_ylim([ymin, ymax])
        ax.set_xlim([xmin, xmax])
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_aspect('equal')

        # Plot pixels used to select endmembers
        for i in range(0,len(x)):
            ax.scatter(centroid[i].transpose()[0], centroid[i].transpose()[1], color=list(cmap(cmap_idx[i])), s=0.1, alpha = 0.2)

        f.tight_layout() 
        plt.axis('off')
        plt.savefig('../png/figure_3Embeddings_Tile%02d_%s.png' % (tile_idx,umetric),dpi=300)
        plt.show()
        

Duration for UMAP fitting: 0:02:01.162803


NameError: name 'peak_local_max' is not defined